# Proyecto de Visión Artificial y Reconocimiento de Patrones

## Reconocimiento Visual de Videojuegos
* Pablo
* Alberto Pérez Álvarez

## 1. Introducción

En esta práctica aplicaremos **Deep Learning** para clasificar imágenes satelitales del conjunto de datos **EuroSAT**. El objetivo es identificar el uso del suelo (bosques, zonas industriales, cultivos, etc.) utilizando tanto redes diseñadas desde cero como técnicas de Transfer Learning.

### Instrucciones de Dataset (Kaggle)
Para que esta libreta funcione, debes añadir el siguiente dataset a tu entorno (en principio, si clonais la libreta lo deberíais de tener por defecto):
1.  En el menú derecho, pulsa **Add Input**.
2.  Busca **"EuroSAT Dataset"** (del usuario *Gota Dahiya*).
3.  Añádelo pulsando el botón `+`.

### Configuración Inicial
Asegúrate de tener activada la **GPU** (T4 x2 o P100) para acelerar el entrenamiento.

In [ ]:
#!pip install "protobuf==3.20.3" > /dev/null 2>&1  #Si no, da un error de compatibilidad entre liberías.
import os

# Configuración de entorno
os.environ["TF_USE_LEGACY_KERAS"] = "1"

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.applications import EfficientNetV2B0
from transformers import TFViTModel
from sklearn.utils import class_weight
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import pathlib
import PIL.Image
import time
from sklearn.metrics import classification_report, confusion_matrix
import gc 

In [ ]:
# Reproducibilidad
SEED = 2025
tf.random.set_seed(SEED)
np.random.seed(SEED)

print(f"TensorFlow Version: {tf.__version__}")
print(f"GPU Available: {len(tf.config.list_physical_devices('GPU')) > 0}")

---

## 2. Carga y Gestión de Datos

Utilizaremos `tf.keras.utils.image_dataset_from_directory`. Esta función crea un generador de datos que carga las imágenes del disco bajo demanda, evitando saturar la memoria RAM.

In [ ]:
KAGGLE_PATH = '/kaggle/input/videojuegos/images_dataset'
# Ruta que tendria en local.
LOCAL_PATH = './images_dataset' 

# Kaggle siempre define la variable de entorno 'KAGGLE_KERNEL_RUN_TYPE'
if os.environ.get('KAGGLE_KERNEL_RUN_TYPE') is not None:
    print("Entorno de Kaggle")
    data_dir = KAGGLE_PATH
else:
    data_dir = LOCAL_PATH
    print("Entorno Local:",data_dir)
    
data_dir = pathlib.Path(data_dir)

# Verificación del contenido
all_images = list(data_dir.glob('*/*.jpg'))
image_count = len(all_images)
print(f"Total de imágenes encontradas: {image_count}")

# Verificación de dimensiones reales
first_image = PIL.Image.open(all_images[0])
print(f"Dimensiones reales de una imagen de muestra: {first_image.size}")
print(f"Formato de imagen: {first_image.format}")

# Parámetros Globales
# Ajustamos las constantes al tamaño real detectado (debería ser 64x64)
BATCH_SIZE = 64
IMG_HEIGHT = 64
IMG_WIDTH = 64
"""BATCH_SIZE = 16
IMG_HEIGHT = 480
IMG_WIDTH = 854"""

### 2.1 Creación de Pipelines (`tf.data`) y División Train/Val/Test

Para garantizar la robustez de los resultados, dividiremos los datos de la siguiente manera:
* **70% Entrenamiento:** Para ajustar los pesos de los modelos.
* **15% Validación:** Para ajustar hiperparámetros y *Early Stopping*.
* **15% Test:** Conjunto para la evaluación final.

In [ ]:
def do_pipeline(data_dir, img_height, img_width, batch_size, seed=123):
    """
    Carga el dataset, lo redimensiona y lo divide en Train, Val y Test.
    Retorna: (train_ds, val_ds, test_ds, class_names)
    """
    print(f"--- Cargando dataset con tamaño: {img_height}x{img_width} y Batch: {batch_size} ---")
    
    full_ds = tf.keras.utils.image_dataset_from_directory(
        data_dir,
        seed=seed,
        image_size=(img_height, img_width),
        batch_size=batch_size,
        shuffle=True,
        label_mode='int'
    )
    
    class_names = full_ds.class_names
    print(f"Clases encontradas: {class_names}")
    
    n_batches = tf.data.experimental.cardinality(full_ds).numpy()
    
    train_size = int(0.7 * n_batches)
    val_size = int(0.15 * n_batches)
    test_size = n_batches - train_size - val_size
    
    print(f"Total batches: {n_batches} -> Train: {train_size}, Val: {val_size}, Test: {test_size}")
    
    train_ds = full_ds.take(train_size)
    remaining_ds = full_ds.skip(train_size)
    val_ds = remaining_ds.take(val_size)
    test_ds = remaining_ds.skip(val_size)
    
    return train_ds, val_ds, test_ds, class_names, full_ds

train_ds, val_ds, test_ds, class_names, full_ds = do_pipeline(data_dir,IMG_HEIGHT,IMG_WIDTH,BATCH_SIZE)

### Calculo de pesos para contrarrestar el desbalanceo de datos

In [ ]:
def calculate_class_weights(train_ds):
    """
    Extrae las etiquetas del dataset de entrenamiento y calcula los pesos
    para equilibrar las clases durante el entrenamiento.
    """
    print("Calculando pesos de las clases ")
    
    # Extraemos todas las etiquetas (y) del dataset
    # Iteramos sobre el dataset y concatenamos solo las etiquetas (y)
    train_labels = np.concatenate([y.numpy() for x, y in train_ds], axis=0)
    
    classes = np.unique(train_labels)
    
    weights = class_weight.compute_class_weight(
        class_weight='balanced',
        classes=classes,
        y=train_labels
    )
    
    # Convertimos a un diccionario {indice_clase: peso} que es lo que pide Keras
    class_weight_dict = dict(zip(classes, weights))
    
    print("Pesos calculados:")
    for cls, weight in class_weight_dict.items():
        print(f"  Clase {cls}: {weight:.4f}")
        
    return class_weight_dict

class_weights = calculate_class_weights(train_ds)

### 2.2 Optimización (Performance)
Usamos `.cache()` y `.prefetch()` para que la GPU no tenga que esperar a la carga de datos.

In [ ]:
def do_performance(train_ds, val_ds, test_ds):
    AUTOTUNE = tf.data.AUTOTUNE
    
    train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
    val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
    test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)
    
    return train_ds, val_ds, test_ds

train_ds, val_ds, test_ds = do_performance(train_ds, val_ds, test_ds)

### 2.3 Análisis Exploratorio (EDA)
En esta práctica no nos vamos a centar en el EDA y preprocesamiento. Solo se piden (obligatoriamente) las siguientes tareas:

**Tarea:**
1.  Visualiza una cuadrícula de imágenes del conjunto de entrenamiento con sus etiquetas (preferiblemente mostrando alguna de cada clase).
2.  Analiza si el dataset está balanceado mostrando un gráfico de barras con la cantidad de imágenes por clase.

In [ ]:
plt.figure(figsize=(15, 10))

# 1. Usamos 'unbatch()' para sacar las imágenes de los paquetes
# 2. Usamos 'take(25)' para coger exactamente las que necesitamos
# 3. Usamos enumerate para saber en qué posición (i) del subplot estamos
for i, (image, label) in enumerate(train_ds.unbatch().take(25)):
    
    ax = plt.subplot(5, 5, i + 1)
    
    # Ya no necesitamos [i] porque 'image' es una sola foto, no un lote
    plt.imshow(image.numpy().astype("uint8"))
    
    # Manejo del label
    # Nota: Dependiendo de tu dataset, label podría ser un escalar o un array de 1 elemento
    label_index = int(label) 
    plt.title(class_names[label_index])
    plt.axis("off")

plt.suptitle("Imagenes de ejemplo del dataset", fontsize=16)
plt.tight_layout()
plt.show()


class_counts = {name: 0 for name in class_names}
for _, labels in full_ds:
    for label in labels:
        class_name = class_names[int(label)]
        class_counts[class_name] += 1

# Lo hacemos con un dataframe, pues es mas facil hacer el plot.
df_counts = pd.DataFrame(list(class_counts.items()), columns=['Class', 'Count'])

plt.figure(figsize=(12, 6))
bplot = sns.barplot(x='Count', y='Class', data=df_counts, palette='viridis', hue='Class')
# tenemos que quitar la leyenda manualmente porque da error con legend=False
if bplot.get_legend() is not None:
    bplot.get_legend().remove()

plt.title('Distribucion de imagenes en el dataset', fontsize=16)
plt.xlabel('Numero de imagenes', fontsize=12)
plt.ylabel('Clase', fontsize=12)
plt.grid(axis='x', linestyle='--', alpha=0.7)

# añadimos el numero al final de la barra para verlo mejor
for index, value in enumerate(df_counts['Count']):
    plt.text(value + 50, index, str(value), va='center')

plt.tight_layout()
plt.show()

mean_count = df_counts['Count'].mean()
std_count = df_counts['Count'].std()
print(f"Media de imagenes: {mean_count:.2f}")
print(f"Desviacion estandar: {std_count:.2f}")

Como podemos observar en el grafico de barras, y confirmado por la media y la desviación, existe un claro desbalance entre los datos, aunque no hay ninguna clase con muchos más datos. Sin embargo la clase Pasture cuenta con bastantes menos datos que las máximas.Esto nos condicionará a la hora de entrenar pues habrá que tener en cuenta aquellas clases que no cuenten con tantos ejemplos.

---
## 3. Baseline: Perceptrón Multicapa (MLP)

Antes de usar redes convolucionales, diseñaremos una red densa clásica (`Sequential`) como baseline para comprobar las limitaciones de tratar imágenes como datos tabulares planos.

**Requisitos:**
1.  La red debe comenzar normalizando los valores de entrada ($0-1$) y aplanando la imagen para convertir la matriz 3D en un vector 1D.
2.  Eres libre de elegir la profundidad y anchura de la red. Ten en cuenta que la entrada tiene más de $12.000$ dimensiones ($64 \times 64 \times 3$).
3.  Configura correctamente la última capa para un problema de clasificación de 10 clases.
4.  Compila el modelo utilizando el optimizador `adam` y la función de pérdida `sparse_categorical_crossentropy`.
5.  Se recomienda guardar el **tiempo de entrenamiento**, en este y todos los modelos, para posibles comparaciones posteriores.
6.  Guarda el mejor modelo generado como `best_mlp.keras`.

**Evaluación:**
1.  Utiliza `accuracy` como métrica de monitorización. Pensad por qué podemos usar esta métrica.
2.  Genera y analiza las gráficas de `loss` y `accuracy` (Entrenamiento vs. Validación).
3.  **Pregunta:** ¿Por qué, teóricamente, un MLP no es la arquitectura ideal para este problema comparado con una CNN?

In [ ]:
# Funcion que usamos para imprimir las graficas de los distintos modelos
def plot_history(history, model_name, stop_epoch=None):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    f1 = history.history['f1_score']
    val_f1 = history.history['val_f1_score']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    
    epochs_range = range(len(acc)) 
    
    plt.figure(figsize=(12, 10))
    
    plt.subplot(2, 2, 1)
    plt.plot(epochs_range, acc, label='Training accuracy')
    plt.plot(epochs_range, val_acc, label='Validation accuracy')
    if stop_epoch is not None:
        plt.axvline(x=stop_epoch, color='r', linestyle='--', label=f'Early Stopping (Ep {stop_epoch+1})')
    plt.legend(loc='lower right')
    plt.title(f'{model_name}: Accuracy')
    plt.grid(True)

    plt.subplot(2, 2, 3)
    plt.plot(epochs_range, f1, label='Training f1')
    plt.plot(epochs_range, val_f1, label='Validation f1')
    if stop_epoch is not None:
        plt.axvline(x=stop_epoch, color='r', linestyle='--', label=f'Early Stopping (Ep {stop_epoch+1})')
    plt.legend(loc='lower right')
    plt.title(f'{model_name}: F1-score')
    plt.grid(True)
    
    plt.subplot(2, 2, 2)
    plt.plot(epochs_range, loss, label='Pérdida de training')
    plt.plot(epochs_range, val_loss, label='Perdida de validation')
    plt.legend(loc='upper right')
    plt.title(f'{model_name}: Perdida')
    plt.grid(True)
    
    plt.tight_layout() # Para que se muestren bien con márgenes

    plt.show()

In [ ]:
class SparseF1Score(tf.keras.metrics.F1Score):
    def update_state(self, y_true, y_pred, sample_weight=None):
       
        num_classes = tf.shape(y_pred)[-1]  # numero de clases de la predicción
        y_true = tf.reshape(y_true, [-1])
        y_true_one_hot = tf.one_hot(tf.cast(y_true, tf.int32), num_classes) # Convertimos los enteros (sparse) a One-Hot
        
        super().update_state(y_true_one_hot, y_pred, sample_weight) # Llamamos a la función original con los datos corregidos

In [ ]:
def build_mlp_model():
    model = models.Sequential([
        layers.InputLayer(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)), # El input es alto, ancho y n canales de RGB
        
        # Normalización de las capas (0-255 -> 0-1)
        layers.Rescaling(1./255),
        # 64 * 64 * 3 = 12,288 dimensiones, pero aplanando lo dejamos como un vector
        layers.Flatten(),
        
        # Las capas densas estaran conectadas unas entre otras
        layers.Dense(512, activation='relu'), # rectified linear unit
        layers.Dense(256, activation='relu'), 
        layers.Dense(128, activation='relu'),
        
        # Hay 10 posibles outputs
        layers.Dense(10, activation='softmax') # devuelve probabilidades para cada clase
    ], name="MLP_Baseline")
    
    return model

mlp_model_ES = build_mlp_model()
mlp_model_ES.summary()

mlp_model_ES.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy', SparseF1Score(average='weighted', name='f1_score')]

)

callbacks_list_mlp = [
    callbacks.EarlyStopping( 
        monitor='val_loss', 
        patience=10, 
        restore_best_weights=True,
        verbose=1
    ),
    callbacks.ModelCheckpoint(
        filepath='best_mlp.keras',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
]

print("\nMLP con Early Stopping")
start_time = time.time()

history_mlp_ES = mlp_model_ES.fit(
    train_ds,
    epochs=50, # lo ponemos alto y lo parara el early stop.
    validation_data=val_ds,
    callbacks=callbacks_list_mlp
)

end_time = time.time()
mlp_ES_training_time = end_time - start_time
print(f"\Tiempo entrenamiento MLP: {mlp_ES_training_time:.2f} s")

epoch_parada_MLP = len(history_mlp_ES.history['loss']) - 1

#### Sin Early Stopping para comparar entre modelos
Originalmente únicamente lo ejecutábamos con Early Stopping, pero con el fin de visualizar mejor las gráficas y una mejor comparación con otros modelos (a igualdad de épocas), hemos decidido volver a entrenarlo sin Early Stopping.

In [ ]:
mlp_model = build_mlp_model()
#mlp_model.summary()

mlp_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy', SparseF1Score(average='weighted', name='f1_score')]

)

callbacks_list_mlp_no_ES = [
    callbacks.ModelCheckpoint(
        filepath='best_mlp.keras',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
]

print("\nMLP")
start_time = time.time()

history_mlp = mlp_model.fit(
    train_ds,
    epochs=50, 
    validation_data=val_ds,
    callbacks=callbacks_list_mlp_no_ES
)

end_time = time.time()
mlp_training_time = end_time - start_time
print(f"\Tiempo entrenamiento MLP: {mlp_training_time:.2f} s")

#### Gráficas de accuracy y perdida

In [ ]:
plot_history(history_mlp, "MLP Baseline", epoch_parada_MLP)

#### Tabla comparativa

In [ ]:
mlp_val_acc = max(history_mlp.history['val_accuracy'])
mlp_val_f1 = max(history_mlp.history['val_f1_score'])
mlp_time = mlp_training_time

mlpES_val_acc = max(history_mlp_ES.history['val_accuracy'])
mlpES_val_f1 = max(history_mlp_ES.history['val_f1_score'])
mlpES_time = mlp_ES_training_time

print("\n" + "="*50)
print("      COMPARATIVA: MLP con vs sin Early Stopping")
print("="*40)
print(f"{'Métrica':<20} | {'MLP':<15} | {'MLP con ES':<15}")
print("-" * 56)
print(f"{'Tiempo (s)':<20} | {mlp_time:.1f}            | {mlpES_time:.1f}")
print(f"{'Mejor Accuracy':<20} | {mlp_val_acc:.4f}          | {mlpES_val_acc:.4f}")
print(f"{'Mejor F1-Score':<20} | {mlp_val_f1:.4f}          | {mlpES_val_f1:.4f}")

#### Evaluación
1. Como el desbalance de datos no es muy grave (ninguna clase tiene la gran mayoría de datos), el uso de accuracy no debería ser demasiado perjudicial. Además se puede ver como la gráfica del f1 y la de accuracy son prácticamente iguales.


2. Podemos ver cómo al principio el modelo generaliza muy bien, llegando a superar el accuracy de validation al de training. Sin embargo, a partir de la época 20 (y más aún en la 25) se aprecia un bajón en el accuracy de validation, posiblemente provocado porque el modelo está sobreajustando y aprendiéndose de memoria el dataset. Esta hipótesis se hace más fuerte al comprobar que la pérdida para validation también sufre de picos sobre la época 20 y se mantiene elevada hasta el final.


3. Al aplanar la imágen perdemos información sobre el posicionamiento de cada píxel, que pasan de estar en una matriz  (con vecinos en dos dimensiones), a un vector (solo vecinos en dos lados, una dimensión). Esto provoca que el MLP no pueda aprender formas o texturas. Una CNN puede conservar la matriz 2D, por lo que no cuenta con esta limitación.

    Además, un MLP no tiene resistencia al cambio de posición de objetos en las imágenes, por lo que si aprende lo que es un río que aparece en el lado superior de la imagen, tendrá que volver a aprenderlo para otra imágen en el que salga en la parte inferior. Nuevamente, una CNN no comparte esta limitación, pues tiene la capacidad de reconocer patrones a lo largo de la imagen, no únicamente en una posición.

---
## 4. Arquitectura Propia: Red Neuronal Convolucional (CNN)

Ahora diseñarás una arquitectura adaptada específicamente para el procesamiento de imágenes. Tu objetivo es **superar el rendimiento del Baseline (MLP)** aprovechando la capacidad de las CNN para aprender patrones espaciales y texturas.

Eres libre de definir la profundidad de la red y el número de filtros, pero debes cumplir los siguientes **requisitos**:
1.  **Data Augmentation:** Incluye capas de preprocesamiento al inicio (ej. `RandomFlip`, `RandomRotation`) para mejorar la generalización.
2.  **Estructura:** Se recomienda usar bloques repetitivos de `Conv2D` $\rightarrow$ `MaxPooling2D`.
    * *Advertencia:* Las imágenes son de $64 \times 64$. Si abusas de las capas de *pooling*, reducirás la imagen a $1 \times 1$ antes de tiempo, perdiendo información.
3.  **Regularización:** Es obligatorio implementar `Dropout` y/o `BatchNormalization` para reducir el sobreajuste. Experimenta con su posición y valores.
4.  **Callbacks:** Configura `EarlyStopping` y `ModelCheckpoint`.

**Tarea:**
1.  Diseña, compila y entrena tu modelo.
2.  Compara las curvas de aprendizaje con las del MLP. ¿Converge más rápido? ¿Sufre menos sobreajuste?
3.  Guarda el mejor modelo generado como `best_cnn.keras`.
4.  Compara el ratio entre tamaño del modelo y accuracy del modelo generado (CNN) con el MLP anterior.
5.  Se valorará el estudio de la influencia del aumento de datos y del uso de `Dropout` y/o `BatchNormalization`.

In [ ]:
BATCH_SIZE = 64
IMG_HEIGHT = 224
IMG_WIDTH = 224

train_ds, val_ds, test_ds, class_names, full_ds = do_pipeline(data_dir,IMG_HEIGHT,IMG_WIDTH,BATCH_SIZE)
train_ds, val_ds, test_ds = do_performance(train_ds, val_ds, test_ds)

In [ ]:
def build_cnn_model():
    # evitamos memorizar el dataset con data augmentation 
    # Giramos y hacemos zoom aleatorio a las imagenes mientras entrena
    data_augmentation = models.Sequential([
        layers.RandomFlip("horizontal_and_vertical"),
        layers.RandomRotation(0.2),
        layers.RandomZoom(0.1),
    ], name="data_augmentation")
    
    inputs = layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3)) # 64x64 pixeles, 3 canales RGB
    
    x = data_augmentation(inputs)

    # 0-255 a 0-1.
    x = layers.Rescaling(1./255)(x)

    # Convoluciones
    
    # Buscamos detalles simples como bordes o esquinas.
    x = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x) # Esto ayuda a estabilizar el aprendizaje
    x = layers.MaxPooling2D((2, 2))(x) # Reducimos la imagen a la mitad, 32x32

    # Buscamos texturas o formas más definidas.
    x = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x) # Bajamos a 16x16

    x = layers.Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x) # Bajamos a 8x8

    x = layers.Conv2D(256, (3, 3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x) # Bajamos a 4x4

    # Los datos a un vector 
    x = layers.Flatten()(x)
    x = layers.Dense(512, activation='relu')(x)
    
    # Hacemos dropout para apagar la mitad de las neuronas al azar y evitar overfitting
    x = layers.Dropout(0.5)(x) 
    
    # 10 outputs
    outputs = layers.Dense(10, activation='softmax')(x)

    model = keras.Model(inputs=inputs, outputs=outputs, name="Custom_CNN")
    return model

cnn_model_ES = build_cnn_model()
cnn_model_ES.summary()

cnn_model_ES.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy', SparseF1Score(average='weighted', name='f1_score')]
)

callbacks_list_cnn = [
    callbacks.EarlyStopping(
        monitor='val_loss',
        patience=10, 
        restore_best_weights=True,
        verbose=1
    ),
    callbacks.ModelCheckpoint(
        filepath='best_cnn.keras',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=0 
    )
]

print("\nCNN")
start_time_cnn = time.time()

history_cnn_ES = cnn_model_ES.fit(
    train_ds,
    epochs=50, # no deberia llegar a hacer los 50 por el early stop
    validation_data=val_ds,
    callbacks=callbacks_list_cnn,
    verbose=1
)

end_time_cnn = time.time()
cnn_ES_training_time = end_time_cnn - start_time_cnn
print(f"\nTiempo de entrenamiento CNN: {cnn_ES_training_time:.2f} s")
epoch_parada_CNN = len(history_cnn_ES.history['loss']) - 1

#### Sin Early Stopping

In [ ]:
callbacks_list_cnn_no_ES = [
    callbacks.ModelCheckpoint(
        filepath='best_cnn.keras',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=0 
    )
]

cnn_model = build_cnn_model()
cnn_model.summary()

cnn_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy', SparseF1Score(average='weighted', name='f1_score')]
)

print("\nCNN sin Early Stopping")
start_time_cnn = time.time()

history_cnn = cnn_model.fit(
    train_ds,
    epochs=50, 
    validation_data=val_ds,
    callbacks=callbacks_list_cnn_no_ES,
    verbose=1
)

end_time_cnn = time.time()
cnn_training_time = end_time_cnn - start_time_cnn
print(f"\nTiempo de entrenamiento CNN sin Early Stop: {cnn_training_time:.2f} s")


In [ ]:
# COMPARACION DE EFICIENCIA ENTRE CNN Y MLP
# Intentamos recuperar los datos del MLP anterior para comparar.
# Si no existen (por no haber corrido esa celda), ponemos ceros para que no falle el código.
try:
    mlp_val_acc = max(history_mlp.history['val_accuracy'])
    mlp_val_f1 = max(history_mlp.history['val_f1_score'])
    mlp_time = mlp_model.count_params()
except NameError:
    mlp_val_acc = 0.0 
    mlp_val_f1 = 0.0
    mlp_time = 0    

cnn_val_acc = max(history_cnn.history['val_accuracy'])
cnn_val_f1 = max(history_cnn.history['val_f1_score'])
cnn_params = cnn_model.count_params()

print("\n" + "="*40)
print("      COMPARATIVA: MLP vs CNN")
print("="*40)
print(f"{'Métrica':<20} | {'MLP':<15} | {'CNN Propia':<15}")
print("-" * 56)
print(f"{'Nº Parámetros':<20} | {mlp_time:<15,} | {cnn_params:<15,}")
print(f"{'Mejor Accuracy':<20} | {mlp_val_acc:.4f}          | {cnn_val_acc:.4f}")
print(f"{'Mejor F1-Score':<20} | {mlp_val_f1:.4f}          | {cnn_val_f1:.4f}")


if cnn_params > 0 and mlp_time > 0:
    ratio_mlp = mlp_time / mlp_val_acc
    ratio_cnn = cnn_params / cnn_val_acc
    print("-" * 56)
    print(f"{'Eficiencia':<20} | {int(ratio_mlp):<15,} | {int(ratio_cnn):<15,} (menos mejor)")

#### Comparación entre CNN y MLP

Como vemos en la tabla, la CNN cuenta con muchos menos parámetros y un accuracy bastante superior. Esto nos lleva a una mucha mejor eficiencia para el CNN.

#### Gráficas de accuracy y perdida

In [ ]:
plot_history(history_cnn, "Custom CNN", epoch_parada_CNN)

#### Evaluación
Las gráficas demuestran que nuestra CNN sigue haciendo un poco de overfitting, pues la validación sigue estando por debajo del training. Sin embargo, esta vez la diferencia entre el validation y el training es constante a lo largo de las épocas. También podemos observar que el validation tiene muchos picos. Pensamos que esto ocurre por existir lotes de imágenes más "confusas" y otras más fáciles para el modelo. En el train esto no afecta tanto pues las imágenes se modifican con el data augmentation, pero  el validation al usar imágenes "limpias" es más sensible a este efecto. 

En adición, comparando con el MLP anterior vemos como la CNN ya obtiene mejores resultados en sus primeras épocas, por lo que aprende más características más útiles y más rápido. Además la CNN tiene un sobreajuste que no aumenta con el tiempo, como sí ocurría con el MLP.

### Estudio de Data Augmentation, Normalizacion y Dropout

#### Sin Data Augmentation

In [ ]:
def build_cnn_model_without_DA():
    # evitamos memorizar el dataset con data augmentation 
    # Giramos y hacemos zoom aleatorio a las imagenes mientras entrena
    
    inputs = layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3)) # 64x64 pixeles, 3 canales RGB
    
    x = inputs

    # 0-255 a 0-1.
    x = layers.Rescaling(1./255)(x)

    # Convoluciones
    
    # Buscamos detalles simples como bordes o esquinas.
    x = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x) # Esto ayuda a estabilizar el aprendizaje
    x = layers.MaxPooling2D((2, 2))(x) # Reducimos la imagen a la mitad, 32x32

    # Buscamos texturas o formas más definidas.
    x = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x) # Bajamos a 16x16

    x = layers.Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x) # Bajamos a 8x8

    x = layers.Conv2D(256, (3, 3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x) # Bajamos a 4x4

    # Los datos a un vector 
    x = layers.Flatten()(x)
    x = layers.Dense(512, activation='relu')(x)
    
    # Hacemos dropout para apagar la mitad de las neuronas al azar y evitar overfitting
    x = layers.Dropout(0.5)(x) 
    
    # 10 outputs
    outputs = layers.Dense(10, activation='softmax')(x)

    model = keras.Model(inputs=inputs, outputs=outputs, name="Custom_CNN")
    return model

cnn_model = build_cnn_model_without_DA()
cnn_model.summary()

cnn_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy', SparseF1Score(average='weighted', name='f1_score')]
)

callbacks_list_cnn = [
    callbacks.EarlyStopping(
        monitor='val_loss',
        patience=10, 
        restore_best_weights=True,
        verbose=1
    )
]

print("\nCNN sin DA")
start_time_cnn_w_DA = time.time()

history_cnn_without_DA = cnn_model.fit(
    train_ds,
    epochs=50, # no deberia llegar a hacer los 50 por el early stop
    validation_data=val_ds,
    callbacks=callbacks_list_cnn,
    verbose=1
)

end_time_cnn = time.time()
cnn_training_time_w_DA = end_time_cnn - start_time_cnn_w_DA
print(f"\nTiempo de entrenamiento CNN: {cnn_training_time_w_DA:.2f} s")

#### Sin Normalizacion

In [ ]:
def build_cnn_model_without_N():
    # evitamos memorizar el dataset con data augmentation 
    # Giramos y hacemos zoom aleatorio a las imagenes mientras entrena
    data_augmentation = models.Sequential([
        layers.RandomFlip("horizontal_and_vertical"),
        layers.RandomRotation(0.2),
        layers.RandomZoom(0.1),
    ], name="data_augmentation")
    
    inputs = layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3)) # 64x64 pixeles, 3 canales RGB
    
    x = data_augmentation(inputs)

    # 0-255 a 0-1.
    x = layers.Rescaling(1./255)(x)

    # Convoluciones
    
    # Buscamos detalles simples como bordes o esquinas.
    x = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x) # Esto ayuda a estabilizar el aprendizaje
    x = layers.MaxPooling2D((2, 2))(x) # Reducimos la imagen a la mitad, 32x32

    # Buscamos texturas o formas más definidas.
    x = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x) # Bajamos a 16x16

    x = layers.Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x) # Bajamos a 8x8

    x = layers.Conv2D(256, (3, 3), padding='same', activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x) # Bajamos a 4x4

    # Los datos a un vector 
    x = layers.Flatten()(x)
    x = layers.Dense(512, activation='relu')(x)
    
    # Hacemos dropout para apagar la mitad de las neuronas al azar y evitar overfitting
    x = layers.Dropout(0.5)(x) 
    
    # 10 outputs
    outputs = layers.Dense(10, activation='softmax')(x)

    model = keras.Model(inputs=inputs, outputs=outputs, name="Custom_CNN")
    return model

cnn_model = build_cnn_model_without_N()
cnn_model.summary()

cnn_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy', SparseF1Score(average='weighted', name='f1_score')]
)

callbacks_list_cnn = [
    callbacks.EarlyStopping(
        monitor='val_loss',
        patience=10, 
        restore_best_weights=True,
        verbose=1
    )
]

print("\nCNN")
start_time_cnn = time.time()

history_cnn_without_normalization = cnn_model.fit(
    train_ds,
    epochs=50, # no deberia llegar a hacer los 50 por el early stop
    validation_data=val_ds,
    callbacks=callbacks_list_cnn,
    verbose=1
)

end_time_cnn = time.time()
cnn_training_time = end_time_cnn - start_time_cnn
print(f"\nTiempo de entrenamiento CNN: {cnn_training_time:.2f} s")

#### Sin dropout

In [ ]:
def build_cnn_model_without_dropout():
    # evitamos memorizar el dataset con data augmentation 
    # Giramos y hacemos zoom aleatorio a las imagenes mientras entrena
    data_augmentation = models.Sequential([
        layers.RandomFlip("horizontal_and_vertical"),
        layers.RandomRotation(0.2),
        layers.RandomZoom(0.1),
    ], name="data_augmentation")
    
    inputs = layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3)) # 64x64 pixeles, 3 canales RGB
    
    x = data_augmentation(inputs)

    # 0-255 a 0-1.
    x = layers.Rescaling(1./255)(x)

    # Convoluciones
    
    # Buscamos detalles simples como bordes o esquinas.
    x = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x) # Esto ayuda a estabilizar el aprendizaje
    x = layers.MaxPooling2D((2, 2))(x) # Reducimos la imagen a la mitad, 32x32

    # Buscamos texturas o formas más definidas.
    x = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x) # Bajamos a 16x16

    x = layers.Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x) # Bajamos a 8x8

    x = layers.Conv2D(256, (3, 3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x) # Bajamos a 4x4

    # Los datos a un vector 
    x = layers.Flatten()(x)
    x = layers.Dense(512, activation='relu')(x)    
    
    # 10 outputs
    outputs = layers.Dense(10, activation='softmax')(x)

    model = keras.Model(inputs=inputs, outputs=outputs, name="Custom_CNN")
    return model

cnn_model = build_cnn_model_without_dropout()
cnn_model.summary()

cnn_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy', SparseF1Score(average='weighted', name='f1_score')]
)

callbacks_list_cnn = [
    callbacks.EarlyStopping(
        monitor='val_loss',
        patience=10, 
        restore_best_weights=True,
        verbose=1
    )
]

print("\nCNN")
start_time_cnn = time.time()

history_cnn_without_dropout = cnn_model.fit(
    train_ds,
    epochs=50, # no deberia llegar a hacer los 50 por el early stop
    validation_data=val_ds,
    callbacks=callbacks_list_cnn,
    verbose=1
)

end_time_cnn = time.time()
cnn_training_time = end_time_cnn - start_time_cnn
print(f"\nTiempo de entrenamiento CNN: {cnn_training_time:.2f} s")

#### Gráficas

In [ ]:
#Original para comparar
plot_history(history_cnn_ES, "Custom CNN")

In [ ]:
plot_history(history_cnn_without_DA, "Custom CNN sin Data Augmentation")


Vemos que al eliminar el data augmentation (y por tanto las rotaciones y los zooms aleatorios), la red se aprende de memoria las características en vez de generalizar, lo que lleva a un mayor puntuaje en training y menor en validation (overfitting). Además podemos ver que los picos han aumentado, posiblemente a causa del gran overfitting que tiene el modelo sin aplicar un data augmentation.

In [ ]:
plot_history(history_cnn_without_normalization, "Custom CNN sin Normalización")

Vemos que sin aplicar Batch Normalization el modelo se vuelve muy inestable en sus predicciones de validation. Esto puede deberse a que los pesos de la red cambian demasiado bruscamente, haciendo que el gradiente sea inestable. En adición, el modelo tarda más en aprender, posiblemente por estos cambios tan bruscos de parámetros.

In [ ]:
plot_history(history_cnn_without_dropout, "Custom CNN sin Dropout")

In [ ]:
plot_history(history_cnn, "Custom CNN sin Early Stopping", epoch_parada_CNN)

Con estas gráficas podemos validar que el no desactivar neuronas provoca un mayor overfitting (pues hay mayor separación entre validation y training), ya que el modelo se fija más en ciertas características en vez de generalizar. Por otro lado, podemos observar que al activar Early Stopping, el modelo no habria llegado a un estado en el que es mas preciso, a coste de un mayor overfitting.

---
## 5. Transfer Learning (Estado del Arte)

En esta sección, utilizaremos una red preentrenada en ImageNet para aprovechar el conocimiento extraído de millones de imágenes. Dado el tamaño de entrada ($64 \times 64$), buscamos arquitecturas eficientes que no reduzcan excesivamente la dimensionalidad espacial.

Debes construir un nuevo modelo siguiendo estas **pautas**:
1.  Integra la misma capa de aumento de datos definida en la sección anterior.
2.  Importa una arquitectura robusta (como `ResNet50V2` o `EfficientNetV2B0`) con pesos de `imagenet`. Recuerda configurar `include_top=False` y definir el `input_shape` correcto.
3.  Congela los pesos del modelo base (Feature Extraction) y añade tu(s) propia(s) capa(s) completamente contectada(s) para clasificación. Se recomienda usar `GlobalAveragePooling2D` para conectar la base convolucional con la capa de salida.

**Tarea:**
1.  Construye, compila y entrena el modelo.
2.  Guarda el mejor modelo generado como `best_transfer.keras`.
3.  Analiza si el uso de modelos preentrenados justifica la mejora en precisión respecto a tu CNN "artesanal", considerando el tiempo de entrenamiento y el número de parámetros.
4.  Se valorará el estudio de descongelar los últimos bloques del modelo base para realizar un *Fine-Tuning* con una tasa de aprendizaje reducida (ej. $1e-5$).

#### Función genérica para agregar más modelos

In [ ]:
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.1),
])

def build_generic_model(model_name, img_height, img_width, num_classes=10):
    inputs = keras.Input(shape=(img_height, img_width, 3))
    
    x = data_augmentation(inputs)
    
    # Selección del modelo base y Preprocesamiento específico
    if model_name == 'EfficientNetV2B0':
        # EfficientNetV2 normaliza internamente. Espera [0, 255]
        base_model = tf.keras.applications.EfficientNetV2B0(
            include_top=False, weights='imagenet', input_shape=(img_height, img_width, 3)
        )
    
    elif model_name == 'ConvNeXtTiny':
        base_model = tf.keras.applications.ConvNeXtTiny(
            include_top=False, weights='imagenet', input_shape=(img_height, img_width, 3)
        )
        
    elif model_name == 'ResNet50V2':
        # como ResNetV2 espera inputs entre [-1, 1] necesitamos reescalar.
        x = layers.Rescaling(1./127.5, offset=-1)(x)
        base_model = tf.keras.applications.ResNet50V2(
            include_top=False, weights='imagenet', input_shape=(img_height, img_width, 3)
        )
        
    else:
        raise ValueError(f"Modelo {model_name} no soportado")

    # Freezing inicial
    base_model.trainable = False
    
    # training=False para mantener las estadísticas de BatchNormalization de ImageNet
    x = base_model(x, training=False)
    
    # Cabezal de clasificación (Top)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.2)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    model = keras.Model(inputs, outputs, name=f"Transfer_{model_name}")
    return model, base_model

In [ ]:
def train_full_pipeline(model, base_model, train_ds, val_ds, class_weights, early_stop=True ,epochs_warmup=25, epochs_fine=50):
    
    print(f"\n>>> Iniciando entrenamiento para: {model.name}")
    
    # --- PREPARACIÓN DE CALLBACKS ---
    checkpoint_path = f"best_{model.name}.keras"
    
    checkpoint_callback = callbacks.ModelCheckpoint(
        checkpoint_path, 
        monitor='val_accuracy', 
        save_best_only=True, 
        verbose=0
    )
    
    #FEATURE EXTRACTION (Congelado) ---
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy',SparseF1Score(average='weighted', name='f1_score')]
    )

    if (early_stop):
        callbacks_transfer = [
            callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1),
            checkpoint_callback
        ]
    else:
        callbacks_transfer = [
            checkpoint_callback
        ]
    
    print(f"   Congelado (Guardando en {checkpoint_path})...")
    start_time = time.time()
    
    history_tl = model.fit(
        train_ds,
        epochs=epochs_warmup,
        validation_data=val_ds,
        callbacks=callbacks_transfer,
        class_weight=class_weights,
        verbose=1
    )

    fine_tuning_at_epoch = len(history_tl.history['val_accuracy']) - 1
    
    # FINE TUNING 
    print("   Descongelado (Fine Tuning)...")
    base_model.trainable = True

    # Definimos cuántas capas dejar entrenables, si descongelamos todas nos quedamos sin VRAM
    fine_tune_at = len(base_model.layers) - 40

    for layer in base_model.layers[:fine_tune_at]:
        layer.trainable = False
    
    model.compile(
        optimizer=keras.optimizers.Adam(1e-5), # Learning rate muy bajo
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy',SparseF1Score(average='weighted', name='f1_score')]
    )
    
    if (early_stop):
        callbacks_finetune = [
            callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1),
            checkpoint_callback
        ]
    else:
        callbacks_finetune = [
            checkpoint_callback
        ]
    
    history_ft = model.fit(
        train_ds,
        epochs=epochs_fine,
        initial_epoch=history_tl.epoch[-1],
        validation_data=val_ds,
        callbacks=callbacks_finetune,
        class_weight=class_weights,
        verbose=1
    )
    
    total_time = time.time() - start_time
    print(f">>> Tiempo total {model.name}: {total_time:.2f} s")
    
    # Combinamos los hystories de los dos (tl y ft)
    combined_dict = {}
    for key in history_tl.history.keys():
        # get para evitar errores si alguna métrica falta en la segunda fase
        list_1 = history_tl.history.get(key, [])
        list_2 = history_ft.history.get(key, [])
        combined_dict[key] = list_1 + list_2
        
    # Historial, Modelo, Tiempo
    return combined_dict, model, total_time, fine_tuning_at_epoch

### Wrapper para ejecutar los modelos

In [ ]:
model_candidates = [
    'EfficientNetV2B0', 
    'ConvNeXtTiny', 
    'ResNet50V2'
                   ]

# Diccionarios para guardar resultados, tiempos y parámetros para la comparativa
results_ES = {}
times_ES = {}
params_ES = {}
fine_tune_starts_ES = {}
models_ES = {}

print(f"Iniciando entrenamiento de modelos: {model_candidates}")

for model_name in model_candidates:
    print(f"\n--- Procesando modelo: {model_name} ---")
    
    # Construcción del modelo (usamos tu snippet)
    model, base_model = build_generic_model(model_name, IMG_HEIGHT, IMG_WIDTH, num_classes=10)
    
    # Guardamos conteo de parámetros para la tabla
    params_ES[model_name] = model.count_params()
    
    # Entrenamiento
    hist, trained_model, execution_time, fine_tune_start = train_full_pipeline(
        model, 
        base_model, 
        train_ds, 
        val_ds, 
        class_weights
    )
    
    # Guardar resultados
    results_ES[model_name] = hist
    times_ES[model_name] = execution_time
    fine_tune_starts_ES[model_name] = fine_tune_start
    models_ES[model_name] = trained_model
    
    # Limpiar memoria de GPU entre modelos
    keras.backend.clear_session()

#### Transfer Learning sin Early Stopping

In [ ]:
# Diccionarios para guardar resultados, tiempos y parámetros para la comparativa
results = {}
times = {}
params = {}
fine_tune_starts = {}
models = {}

print(f"Iniciando entrenamiento SIN EARLY STOP de modelos: {model_candidates}")

for model_name in model_candidates:
    print(f"\n--- Procesando modelo: {model_name} ---")
    
    # Construcción del modelo (usamos tu snippet)
    model, base_model = build_generic_model(model_name, IMG_HEIGHT, IMG_WIDTH, num_classes=10)
    
    # Guardamos conteo de parámetros para la tabla
    params[model_name] = model.count_params()
    
    # Entrenamiento
    hist, trained_model, execution_time, fine_tune_start = train_full_pipeline(
        model, 
        base_model, 
        train_ds, 
        val_ds, 
        class_weights,
        early_stop = False
    )
    
    # Guardar resultados
    results[model_name] = hist
    times[model_name] = execution_time
    fine_tune_starts[model_name] = fine_tune_start
    models[model_name] = trained_model
    
    # Limpiar memoria de GPU entre modelos
    keras.backend.clear_session()

#### Visualizar graficas y comparativa final

In [ ]:
def plt_transfer_learning(models_history,fine_tune_starts): 
    """
    Recibe un diccionario con:
    - Key: Nombre del modelo (str)
    - Value: Objeto history del entrenamiento
    Ejemplo: plt_transfer_learning({'VGG16': history_vgg, 'ResNet': history_resnet})
    """
    
    # Iteramos sobre cada modelo en el diccionario
    for model_name, history_tl in models_history.items():
        
        # Extraemos las métricas directamente del historial actual
        acc = history_tl['accuracy']
        val_acc = history_tl['val_accuracy']
        loss = history_tl['loss']
        val_loss = history_tl['val_loss']
        f1 = history_tl['f1_score']
        val_f1 = history_tl['val_f1_score']

        plt.figure(figsize=(12, 10))
        
        # --- Gráfica de Accuracy ---
        plt.subplot(2, 2, 1)
        plt.plot(acc, label='Training accuracy')
        plt.plot(val_acc, label='Validation accuracy')
        plt.plot([fine_tune_starts[model_name], fine_tune_starts[model_name]], plt.ylim(), label='Inicio Fine Tuning', ls='--') 
        plt.legend(loc='lower right')
        plt.title(f'Evolución de Accuracy: {model_name}') # Nombre del modelo incluido
        plt.grid(True)

        # --- Gráfica de F1-Score ---
        plt.subplot(2, 2, 3)
        plt.plot(f1, label='Training F1-score')
        plt.plot(val_f1, label='Validation F1-score')
        plt.plot([fine_tune_starts[model_name], fine_tune_starts[model_name]], plt.ylim(), label='Inicio Fine Tuning', ls='--') 
        plt.legend(loc='lower right')
        plt.title(f'Evolución de F1-score: {model_name}') # Nombre del modelo incluido
        plt.grid(True)

        # --- Gráfica de Loss ---
        plt.subplot(2, 2, 2)
        plt.plot(loss, label='Pérdida en training')
        plt.plot(val_loss, label='Pérdida en validation')
        plt.plot([fine_tune_starts[model_name], fine_tune_starts[model_name]], plt.ylim(), label='Inicio Fine Tuning', ls='--')
        plt.legend(loc='upper right')
        plt.title(f'Evolución de la Pérdida: {model_name}') # Nombre del modelo incluido
        plt.grid(True)
        
        plt.tight_layout() # Para que no se solapen textos si hay muchas gráficas
        plt.show()

In [ ]:
# Intentamos recuperar métricas de los modelos anteriores (si existen en memoria)
try:
    mlp_acc = max(history_mlp.history['val_accuracy'])
    cnn_ES_acc = max(history_cnn_ES.history['val_accuracy'])
    cnn_acc = max(history_cnn.history['val_accuracy'])
    mlp_f1 = max(history_mlp.history['val_f1_score'])
    cnn_ES_f1 = max(history_cnn_ES.history['val_f1_score'])
    cnn_f1 = max(history_cnn.history['val_f1_score'])
    mlp_par = mlp_model.count_params()
    cnn_ES_par = cnn_model.count_params()
    cnn_par = cnn_model.count_params()

    mlp_acc_ES = max(history_mlp_ES.history['val_accuracy'])
    mlp_f1_ES = max(history_mlp_ES.history['val_f1_score'])
    mlp_par_ES = mlp_model_ES.count_params()
except NameError:
    # Valores por defecto si no existen
    mlp_acc, cnn_ES_acc, cnn_acc, mlp_f1, cnn_f1_ES, cnn_f1, mlp_par, cnn_par_ES, cnn_par = 0, 0, 0, 0, 0, 0, 0, 0, 0
    mlp_acc_ES, mlp_f1_ES, cnn_ES_f1, mlp_par_ES, cnn_ES_par = 0, 0, 0, 0, 0

print("\n" + "="*85)
print("      COMPARATIVA DE ARQUITECTURAS")
print("="*85)
print(f"{'Modelo':<25} | {'Parámetros':<12} | {'Val Acc':<10} | {'Val F1':<10} | {'Tiempo(s)':<10}")
print("-" * 85)
# Filas de otros modelos 
print(f"{'MLP (Básico)':<25} | {mlp_par:<12,} | {mlp_acc:.4f}   | {mlp_f1:.4f}     | {mlp_training_time if 'mlp_training_time' in locals() else 0:.1f}")
print(f"{'MLP (con Early Stop)':<25} | {mlp_par_ES:<12,} | {mlp_acc_ES:.4f}   | {mlp_f1_ES:.4f}     | {mlp_ES_training_time if 'mlp_ES_training_time' in locals() else 0:.1f}")
print(f"{'CNN':<25} | {cnn_par:<12,} | {cnn_acc:.4f}   | {cnn_f1:.4f}     | {cnn_training_time if 'cnn_training_time' in locals() else 0:.1f}")
print(f"{'CNN (Con Early Stop)':<25} | {cnn_ES_par:<12,} | {cnn_ES_acc:.4f}   | {cnn_ES_f1:.4f}     | {cnn_ES_training_time if 'cnn_ES_training_time' in locals() else 0:.1f}")

print("-" * 85)
# Filas dinámicas para los modelos de Transfer Learning
for name, hist in results_ES.items():
    tl_acc = max(hist['val_accuracy'])
    tl_f1 = max(hist['val_f1_score'])
    tl_par = params_ES[name]
    tl_time = times_ES.get(name, 0)
    
    print(f"{'TL: ' + name+' con ES':<25} | {tl_par:<12,} | {tl_acc:.4f}   | {tl_f1:.4f}     | {tl_time:.1f}")

for name, hist in results.items():
    tl_acc = max(hist['val_accuracy'])
    tl_f1 = max(hist['val_f1_score'])
    tl_par = params_ES[name]
    tl_time = times_ES.get(name, 0)
    
    print(f"{'TL: ' + name:<25} | {tl_par:<12,} | {tl_acc:.4f}   | {tl_f1:.4f}     | {tl_time:.1f}")

print("-" * 85)

#### Gráficas con Early Stopping

In [ ]:
plt_transfer_learning(results_ES,fine_tune_starts_ES)

#### Gráficas SIN Early Stopping

In [ ]:
plt_transfer_learning(results,fine_tune_starts)

La mayor diferencia de estas gráficas frente a nuestra CNN propia es que empieza con un accuracy bastante más alto. Esto puede deberse al hehco de que el modelo ya viene pre-entrenado, y por tanto ya sabe distinguir ciertas formas y texturas. Además no tiene nada de overfitting pues el accuracy de validation es mayor al de train (y su pérdida menor). Al hacer el fine tuning se ve que mejora drásticamente tanto en validation como en train, posiblemente por adaptar los valores previamente congelados a las características de nuestro dataset. Por ultimo, tenemos el mismo caso que antes, al activar la opción Early Stopping el modelo tendra algo menos de precision, sin embargo conforme avanzan las epocas vemos aun más overfitting en el modelo. Antes de que saltara el ES, el modelo no hacia practicamente overfitting, como hemos dicho antes, sin embargo tras el fine tunning y ES combinados el resultado final no sería tan ajustado para otros dataset, dandole menos utilidad real.

Aunque este modelo consigue una mejor accuracy que nuestra CNN, también tarda el doble y usa bastantes más parámetros, por lo que en la mayoría de casos nuestra CNN sería un buen compromiso entre recursos y rendimiento. En comparación con el MLP, este Transfer Learning es mucho mejor, pues sigue teniendo menos que el MLP mientras consigue un rendimiento bastante superior.

---
## 6. (Opcional) Nuevas Fronteras: Vision Transformers (ViT)

Mientras que las CNN han dominado la visión por computador durante una década, los **Transformers** (originalmente diseñados para texto) han irrumpido con fuerza en el campo.

A diferencia de una CNN, que mira píxeles vecinos, un **Vision Transformer (ViT)** divide la imagen en "parches" (por ejemplo, cuadrados de $16 \times 16$) y procesa la relación de cada parche con todos los demás simultáneamente mediante mecanismos de **Atención**.

**Desafío Técnico:**
Los ViT son modelos muy pesados y requieren una resolución de entrada específica (generalmente $224 \times 224$). Además, su integración en Keras requiere adaptadores especiales.

**Tarea (Bonus):**
1.  Instala la librería `transformers` de Hugging Face.
2.  Utiliza la clase `ViTWrapper` proporcionada abajo para cargar un modelo base de Google (`vit-base-patch16-224`).
3.  **Completa la función `build_vit_classifier`**:
    * Añade tu capa de **Data Augmentation** al inicio.
    * Observa cómo adaptamos la imagen (`Resizing`, `Rescaling` y `Permute`) para que sea compatible con el modelo de Google.
    * Añade tu(s) propia(s) capa(s) completamente contectada(s) de clasificación al final (Capas Densas y de Salida).
4.  Entrena por pocas épocas y compara los resultados (y el tiempo de ejecución) con tu mejor CNN.

#### Limpieza de variables y basura para ejecutar en local

In [ ]:
# Limpiamos sesiones anteriores para liberar VRAM de la GPU (en nuestros portátiles es necesario)
tf.keras.backend.clear_session()
gc.collect()

USE_REDUCED_BATCH_SIZE = True 

if USE_REDUCED_BATCH_SIZE:
    BATCH_SIZE_VIT = 16
    print(f"Usando Batch Size REDUCIDO: {BATCH_SIZE_VIT}")
else:
    BATCH_SIZE_VIT = 64
    print(f"Usando Batch Size: {BATCH_SIZE_VIT}")

#### Carga de train y validation

In [ ]:
train_ds_vit = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.15,
    subset="training",
    seed=2025,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE_VIT,
    label_mode='int'
)

val_ds_vit = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.15,
    subset="validation",
    seed=2025,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE_VIT,
    label_mode='int'
)
# Optimización de carga
AUTOTUNE = tf.data.AUTOTUNE
train_ds_vit = train_ds_vit.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds_vit = val_ds_vit.cache().prefetch(buffer_size=AUTOTUNE)

#### Funciones de ViT

In [ ]:
class ViTWrapper(layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        # Cargamos el modelo base pre-entrenado
        # hemos añadido from_pt=True para cargar pesos originales de PyTorch
        # y evitar los errores de lectura en TensorFlow.
        self.vit = TFViTModel.from_pretrained('google/vit-base-patch16-224', from_pt=True)
        self.vit.trainable = False # Congelamos pesos base
        
    def call(self, inputs):
        # El modelo espera 'pixel_values'
        # Retornamos 'pooler_output' que es la representación vectorial de la imagen
        return self.vit(pixel_values=inputs).pooler_output

def build_vit_classifier():
    inputs = keras.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
    
    # 1. Data Augmentation
    x = layers.RandomFlip("horizontal_and_vertical")(inputs)
    x = layers.RandomRotation(0.2)(x)
    x = layers.RandomZoom(0.1)(x)
    
    # 2. Adaptación de Dimensionalidad (Necesario para ViT)
    # Redimensionamos a 224x224 (resolución nativa de ViT)
    x = layers.Resizing(224, 224)(x)
    # Escalamos los píxeles entre -1 y 1
    x = layers.Rescaling(1./127.5, offset=-1)(x)
    # Transponemos canales a (Canales, Alto, Ancho) para compatibilidad con Hugging Face
    x = layers.Permute((3, 1, 2))(x)
    
    # 3. Bloque Transformer
    x = ViTWrapper()(x)
    
    # 4. Clasificación
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.2)(x) # intentamos evitar overfitting
    outputs = layers.Dense(10, activation='softmax')(x)
    
    return keras.Model(inputs, outputs, name="ViT_Classifier")

In [ ]:
vit_model = build_vit_classifier()
vit_model.summary()


vit_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy', SparseF1Score(average='weighted', name='f1_score')]
)

callbacks_vit = [
    callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1),
    callbacks.ModelCheckpoint('best_vit.keras', monitor='val_accuracy', save_best_only=True, verbose=0)
]

print(f"\n--- Iniciando entrenamiento ViT (Batch Size: {BATCH_SIZE_VIT}) ---")
start_time_vit = time.time()


history_vit = vit_model.fit(
    train_ds_vit,
    epochs=10, # Pocas épocas para que tarde menos de 1 hora en local
    validation_data=val_ds_vit,
    callbacks=callbacks_vit
)

end_time_vit = time.time()
vit_training_time = end_time_vit - start_time_vit
print(f"\nTiempo total de entrenamiento (ViT): {vit_training_time:.2f} segundos")

In [ ]:
plot_history(history_vit, "Vision Transformer (ViT)")

# Recuperamos las métricas de la CNN asumiendo que siguen en memoria
cnn_best_val_acc = max(history_cnn.history['val_accuracy'])
cnn_best_val_f1 = max(history_cnn.history['val_f1_score'])
cnn_params = cnn_model.count_params()

# Recuperamos métricas del ViT
vit_best_val_acc = max(history_vit.history['val_accuracy'])
vit_best_val_f1 = max(history_vit.history['val_f1_score'])
vit_params = vit_model.count_params()

# tabla comparativa
print("\n" + "="*75)
print("                         CNN vs ViT")
print("="*75)
print(f"{'Modelo':<25} | {'Parámetros':<12} | {'Val Acc':<10} | {'Val F1':<10} | {'Tiempo(s)':<10}")
print("-" * 75)

# CNN
print(f"{'CNN (Propia)':<25} | {cnn_par:<12,} | {cnn_best_val_acc:.4f}  | {cnn_best_val_f1:.4f}     | {cnn_training_time:.1f}")

# ViT
print(f"{'ViT (HuggingFace)':<25} | {vit_params:<12,} | {vit_best_val_acc:.4f}  | {vit_best_val_f1:.4f}     | {vit_training_time:.1f}")
print("-" * 75)

# Grafica comparativa
plt.figure(figsize=(10, 6))

# Curva CNN
val_acc_cnn = history_cnn.history['val_accuracy']
plt.plot(val_acc_cnn, label=f'CNN Propia (Max: {cnn_best_val_acc:.2%})', linestyle='--', linewidth=2)

# Curva ViT
val_acc_vit = history_vit.history['val_accuracy']
plt.plot(val_acc_vit, label=f'ViT (Max: {vit_best_val_acc:.2%})', linewidth=2)

plt.title('Comparativa de Validación: Convolución vs Transformer')
plt.xlabel('Épocas')
plt.ylabel('Validation Accuracy')
plt.legend(loc='lower right')
plt.grid(True, alpha=0.3)
plt.show()


Vemos que con este dataset en específico la ViT no obtiene apenas ganancia, a pesar de tardar varias veces más que la CNN. Por otro lado, con el mismo número de épocas sí que obtiene bastante mejor resultado, por lo que es posible que si la dejaramos ejecutarse con el mismo número de épocas que la CNN aumentara la diferencia.

Este aumento de tiempo se entiende al mirar el número de parámetros, muy superior a la CNN y MLP. Es posible que con datasets mayores, con más resolución y complejidad el ViT obtuviese muchos mejores resultados.

---
## 7. Evaluación y Análisis Comparativo

En esta sección final evaluaremos el mejor modelo y realizaremos un estudio comparativo de todas las arquitecturas probadas.

**Tareas:**
1.  **Evaluación Cuantitativa:**
    * Carga el mejor modelo de todos los generados.
    * Genera el `classification_report` para ver la precisión, recall y F1-score por cada clase.
    * Visualiza la **Matriz de Confusión**. Identifica los pares de clases más problemáticos.

2.  **Análisis Cualitativo de Errores:**
    * Visualiza algunas imágenes mal clasificadas. 
    * **Requisito:** Muestra la imagen, la etiqueta real, la etiqueta predicha y el **nivel de confianza** (probabilidad) del modelo en esa predicción errónea. ¿Son errores "evitables"?

3.  **Comparativa Final:**
    * Crea un DataFrame de Pandas comparando al menos MLP, CNN Propia y CNN Transfer Learning (opcional ViT). *En este punto se pueden incluir también comparaciones con modelos con/sin aumento de datos, con/sin dropout, diversas arquitecturas...*
    * Ejemplo de columnas: `Modelo`, `Accuracy (Val)`, `F1-Score (Macro)`, `Nº Parámetros`, `Nº Parámetros Entrenables`, `Tiempo de Entrenamiento`.
    * **Conclusión:** Escribe un par de párrafos justificando cuál es la mejor arquitectura considerando el equilibrio entre recursos computacionales (parámetros/tiempo) y precisión.

In [ ]:
def save_metric(model_input, history_input, name, time_input=0):
    """
    Admite nombres de variables (str) o los objetos directamente.
    """
    # 1. OBTENER EL MODELO
    if isinstance(model_input, str):
        if model_input not in globals(): return None
        model = globals()[model_input]
    else:
        model = model_input

    # 2. OBTENER EL HISTORY
    if isinstance(history_input, str):
        if history_input not in globals(): return None
        history_obj = globals()[history_input]
    else:
        history_obj = history_input

    # 3. OBTENER EL TIEMPO
    if isinstance(time_input, str):
        time_val = globals().get(time_input, 0)
    else:
        time_val = time_input

    # 4. EXTRAER MÉTRICAS
    try:
        # Detectamos si es un objeto History de Keras (tiene atributo .history) o un diccionario puro
        history_dict = history_obj.history if hasattr(history_obj, 'history') else history_obj
        
        # Obtenemos máximos
        val_acc = max(history_dict['val_accuracy'])
        val_f1 = max(history_dict['val_f1_score'])
        params = model.count_params()
        
        return {
            "Modelo": name,
            "Accuracy (Val)": val_acc,
            "F1-score (Val)": val_f1,
            "Nº Parámetros": params,
            "Tiempo (s)": time_val
        }
    except Exception as e:
        print(f"Error al procesar métricas de '{name}': {e}")
        return None

In [ ]:
print("Recopilando métricas...")

best_model_path = 'best_transfer_.keras' 
try:
    best_model = tf.keras.models.load_model(
        best_model_path,
        custom_objects={'SparseF1Score': SparseF1Score} 
    )
    print("Modelo cargado.")
except:
    print(f"Error cargando {best_model_path}.")
    best_model = None

metrics_data = []

metrics_data.append(save_metric('mlp_model', 'history_mlp', 'MLP Baseline', 'mlp_training_time'))
metrics_data.append(save_metric('cnn_model', 'history_cnn', 'CNN Propia', 'cnn_training_time'))
for name, hist in results.items():
    metrics_data.append(save_metric(
        model_input=models[name],     
        history_input=hist,              
        name=f"TL: {name}",              
        time_input=times.get(name, 0) 
    ))
    
metrics_data.append(save_metric('vit_model', 'history_vit', 'Vision Transformer', 'vit_training_time'))

# Limpieza de RAM
heavy_vars = ['mlp_model', 'cnn_model', 'transfer_model', 'base_model', 'vit_model',
              'train_ds', 'val_ds', 'train_ds_vit', 'val_ds_vit', 'test_images', 'test_probs']
for var in heavy_vars:
    if var in globals(): del globals()[var]
tf.keras.backend.clear_session()
gc.collect()


# Filtramos 'allBands' si existe
real_class_names = [c for c in class_names if c != 'allBands']
print(f"Clases válidas para evaluación ({len(real_class_names)}): {real_class_names}")

if best_model:
    print("Generando predicciones...")
    y_true, y_pred, error_samples = [], [], []
    
    # Si test_ds fue borrado por el script anterior de limpieza, lo recargamos:
    if 'test_ds' not in globals():
        test_ds = tf.keras.utils.image_dataset_from_directory(
            data_dir, validation_split=0.15, subset="validation", seed=2025,
            image_size=(IMG_HEIGHT, IMG_WIDTH), batch_size=32, label_mode='int'
        )

    for batch_images, batch_labels in test_ds:
        preds = best_model.predict(batch_images, verbose=0)
        preds_ids = np.argmax(preds, axis=1)
        labels_ids = batch_labels.numpy().astype(int)
        
        valid_mask = labels_ids < 10 
        
        y_true.extend(labels_ids[valid_mask])
        y_pred.extend(preds_ids[valid_mask])
        
        if len(error_samples) < 5:
            batch_errors = np.where((preds_ids != labels_ids) & valid_mask)[0]
            for idx in batch_errors:
                if len(error_samples) >= 5: break
                error_samples.append({
                    'img': batch_images[idx].numpy().astype("uint8"),
                    'true': real_class_names[labels_ids[idx]],
                    'pred': real_class_names[preds_ids[idx]],
                    'conf': np.max(preds[idx])
                })
        del batch_images, batch_labels, preds
        gc.collect()

    print("\nMATRIZ DE CONFUSIÓN")
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    # Usamos real_class_names que tiene el tamaño correcto
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=real_class_names, yticklabels=real_class_names)
    plt.ylabel('Real')
    plt.xlabel('Predicción')
    plt.show()
    
    print("\nCLASSIFICATION REPORT")
    print(classification_report(y_true, y_pred, target_names=real_class_names))
    
    if error_samples:
        print(f"\nVisualización de errores:")
        plt.figure(figsize=(15, 4))
        for i, sample in enumerate(error_samples):
            ax = plt.subplot(1, 5, i + 1)
            plt.imshow(sample['img'])
            plt.title(f"R: {sample['true']}\nP: {sample['pred']}\n{sample['conf']:.1%}", 
                      color='red', fontsize=9)
            plt.axis("off")
        plt.show()


df_results = pd.DataFrame([m for m in metrics_data if m is not None])
if not df_results.empty:
    df_results = df_results.sort_values(by="Accuracy (Val)", ascending=False)
    display(df_results)
    
    plt.figure(figsize=(10, 5))
    sns.barplot(data=df_results, x='Modelo', y='Accuracy (Val)', palette='viridis')
    plt.title("Comparativa Final")
    plt.ylim(0, 1.05)
    for i, v in enumerate(df_results['Accuracy (Val)']):
        plt.text(i, v + 0.01, f"{v:.1%}", ha='center')
    plt.show()

    df_results = df_results.sort_values(by="F1-score (Val)", ascending=False)
    display(df_results)
    
    plt.figure(figsize=(10, 5))
    sns.barplot(data=df_results, x='Modelo', y='F1-score (Val)', palette='viridis')
    plt.title("Comparativa Final")
    plt.ylim(0, 1.05)
    for i, v in enumerate(df_results['F1-score (Val)']):
        plt.text(i, v + 0.01, f"{v:.1%}", ha='center')
    plt.show()


### Análisis Cuantitativo
El modelo basado en Transfer Learning se posiciona como la mejor arquitectura. Se ve que supera ampliamente a las otras opciones tanto en accuracy como en F1-score, mostrando un equilibrio muy sólido entre precisión y exhaustividad. Esto confirma que partir de un modelo pre-entrenado permite obtener métricas de rendimiento muy altas, incluso con un conjunto de datos de entrenamiento relativamente pequeño, superando las limitaciones de aprender desde cero.

Con respecto a la matriz de confusion vemos una clara diagonal que nos indica que el modelo predice de forma correcta la mayoria de imagenes. Las mayores confusiones vienen por las parejas de Forest/SeaLake, River/Highway y HerbadeusVegetation/PermanentCrop. El primero de ellos puede deberse a la falta de resolucion de la imagen. En una imagen pequeña si solo cogiesemos el color de la misma, podriamos tener bosques mas azulados o lagos un poco mas verdosos. Cabe señalar que algunas de las imagenes mal clasificadas, son tan dificiles para el modelo como para nosotros, pues con una imagen tan pequeña es dificil obtener el contexto completo.



### Análisis Cualitativo
Al observar las imágenes mal clasificadas, notamos que los fallos no son aleatorios, sino que ocurren en situaciones de ambigüedad visual (como hemos indicado antes, por basar parte de la información en los colores, una recta de color medio gris/azul que puede ser asfalto o agua...). En muchas de estas predicciones erróneas, el modelo asigna un nivel de confianza más bajo que en sus aciertos, lo que sugiere que la red es consciente de la simulitud entre ambas clases.

### Comparativa final
El modelo MLP resultó ineficaz al perder la información espacial (aplanado de imagen), y nuestra CNN, aunque capaz de extraer características, se ha visto muy limitada. El Transfer Learning logró el equilibrio ideal: a pesar de tener una gran cantidad de parámetros totales, el uso de capas congeladas permitió un entrenamiento rápido y una capacidad de generalización muy superior. Por otro lado, el ViT consume muchos recursos pero no llega al nivel del Transfer Learning, por lo que no se justifica (al menos en este dataset) su uso. 

Es por eso que hemos elegido el Transfer Learning como mejor arquitectura, ya que ofrece la máxima precisión con un coste de entrenamiento razonable.

Como nota final, en lo que respecta a la forma de entrenamiento, hemos descubierto que la técnica Early Stopping es muy util si los datos que vamos a predecir cambian. Si el dataset al que tiene que enfrentarse el modelo no cambiase (como en una competición) esto permite un mayor overfitting pero se traduce tambien en mejores resultados (hasta cierto punto). Esto confirma lo que hemos ido viendo en clase a lo largo del cuatrimestre.